In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_269441/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_7_2,0.998918,0.699604,0.999113,0.995877,0.998247,0.000455,0.178328,0.000436,0.000852,0.000644,0.008640,0.021321,0.998068,0.021492,125.392243,281.387748,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
1,model_1_7_3,0.998897,0.698122,0.999013,0.995158,0.997977,0.000464,0.179208,0.000485,0.001000,0.000743,0.008528,0.021530,0.998030,0.021703,125.353306,281.348811,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
2,model_1_7_1,0.998886,0.701280,0.999177,0.996867,0.998567,0.000468,0.177333,0.000405,0.000647,0.000526,0.008883,0.021639,0.998010,0.021813,125.332963,281.328467,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
3,model_1_7_4,0.998828,0.695383,0.998756,0.994140,0.997519,0.000492,0.180834,0.000612,0.001210,0.000911,0.008573,0.022188,0.997908,0.022367,125.232730,281.228235,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
4,model_1_7_5,0.998819,0.695225,0.998728,0.994001,0.997461,0.000496,0.180928,0.000625,0.001239,0.000932,0.008570,0.022274,0.997892,0.022452,125.217416,281.212921,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,model_1_0_4,0.378619,0.221913,0.375146,0.300483,0.342278,0.261088,0.461906,0.213884,0.188574,0.201229,0.310597,0.510967,-0.109610,0.515072,112.685798,268.681302,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
246,model_1_0_3,0.280396,0.153763,0.283767,0.171591,0.234376,0.302358,0.502363,0.245162,0.223320,0.234241,0.333126,0.549871,-0.285008,0.554288,112.392285,268.387790,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
247,model_1_0_2,0.191200,0.095389,0.196153,0.079203,0.144662,0.339836,0.537016,0.275153,0.248226,0.261689,0.354831,0.582955,-0.444286,0.587637,112.158584,268.154089,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
248,model_1_0_1,0.053798,-0.010120,0.080039,-0.065442,0.015983,0.397569,0.599651,0.314898,0.287219,0.301058,0.388048,0.630530,-0.689646,0.635595,111.844776,267.840281,"Hidden Size=[2, 12], regularizer=0.02, learnin..."
